# VLSI DESIGN PROBLEM USING MIXED INTEGER LINEAR PROGRAMMING

""" Team members : vida Zahedi (vida.zahedi@studio.unibo.it) - Samral Tahirli (samral.tahirli@studio.unibo.it) """

In [2]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.1 MB/s eta 0:00:00


In [42]:
#importing requiredlibraries
import time
import math
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations

In [43]:
# read and make the input data from text file
def read_instance(file_name):
     with open(file_name, 'r') as f:
        Width = int(f.readline().strip())
        Number = int(f.readline().strip())
        Pieces = np.array([list(map(int, line.strip().split())) for line in f.readlines()])
     return Width, Number, Pieces


In [51]:
#this function will take the input and will out put the optimized height plus the pplacement of each peice on the plate
def vlsi_optimization(Width, Number, Pieces):
     Widths   = Pieces[:, 0]
     Heights  = Pieces[:, 1]
     Rotation = np.array([[x, y] if x<=y else [y, x] for x, y in zip(Widths, Heights)]) # rotate if needed
     min_height = sum(Widths * Heights) // Width
     max_height = math.ceil(sum(Heights)/(Width//max(Widths)))
     Solver = gp.Model('VLSI-Design')
     M1=10**6
     M2=10**6
     OptHeight = Solver.addVar(name='height', vtype=gp.GRB.INTEGER, lb=min_height, ub=max_height)
     x = Solver.addVars(list(range(Number)), name="x", vtype=gp.GRB.INTEGER, lb=0, ub=Width - Rotation[:, 0].min())
     y = Solver.addVars(list(range(Number)), name="y", vtype=gp.GRB.INTEGER, lb=0, 
                  ub=max_height - Rotation[:, 1].min())
     comb = Solver.addVars([(i,j,k) for (i,j) in combinations(x,2) for k in range(4)], name='comb', vtype=gp.GRB.BINARY)
     c1=Solver.addConstrs((x[i]  + Rotation[i, 0] <= Width for i in range(Number)), name='c1')
     c2 =Solver.addConstrs((y[i] + Rotation[i, 1] <=  OptHeight for i in range(Number)), name='c2')
     c3=Solver.addConstrs((comb.sum(i,j,'*') >= 1 for (i,j) in combinations(x,2)), name = 'c3')
     c4=Solver.addConstrs((x[j]  + Rotation[j, 0] <= x[i]  + M1 * (1 - comb[i,j,0]) for (i,j) in combinations(x,2)), name = 'c4')
     c5=Solver.addConstrs((x[i]  + Rotation[i, 0] <= x[j]  + M1 * (1 - comb[i,j,1]) for (i,j) in combinations(x,2)), name = 'c5')
     c6= Solver.addConstrs((y[j] + Rotation[j, 1] <= y[i]  + M2 * (1 - comb[i,j,2]) for (i,j) in combinations(x,2)), name = 'c6')
     c7= Solver.addConstrs((y[i] + Rotation[i, 1] <= y[j]  + M2 * (1 - comb[i,j,3]) for (i,j) in combinations(x,2)), name = 'c7')
     Solver.setObjective( OptHeight, GRB.MINIMIZE)
     #Solver.setObjective(OptHeight, gp.GRB.MINIMIZE)
     Solver.optimize()
     coords = []
     for i in range(Number):
        coords.append([x[i].x, y[i].x, Rotation[i, 0], Rotation[i, 1]])
     coords = np.array(coords)
     print("Circuit placements:")
     for i in range(Number):
        print("Circuit", i, ": x =", int(coords[i,0]), ", y =", int(coords[i,1]), ", width =", int(coords[i,2]), ", height =", int(coords[i,3]))
     Solver.update()
     return  Solver.objVal,x,y,OptHeight


In [53]:
#solving the instances by calling the functions
Num= 2
start_time = time.time()
file_name = "/content/ins-"+str(Num)+".txt"
Width, Number, Pieces = read_instance(file_name)
Model=vlsi_optimization(Width, Number, Pieces)
#print the execution time 
timing = (time.time() - start_time)
print(timing,"execution-time")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 60 rows, 51 columns and 175 nonzeros
Model fingerprint: 0x5b3332e8
Variable types: 0 continuous, 51 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 9.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 9 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+00, best bound 9.000000000000e+00, gap 0.0000%
Circuit placements:
Circuit 0 : x = 6 , y = 0 , width = 3 , height = 3
Circuit 1 : x = 3 , y = 5 , width = 3 , height = 4
Circuit 2 : x = 3 , y = 0 , width = 3 , height = 5
Circuit 3 : x = 6